In [42]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_unet(input_shape=(256, 256, 1)):
    inputs = tf.keras.Input(input_shape)

    # Downsampling
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Upsampling
    u1 = layers.UpSampling2D((2, 2))(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = layers.UpSampling2D((2, 2))(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = models.Model(inputs, outputs)
    return model


In [43]:
def build_regression(input_shape=(256, 256, 1)):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(3, activation='linear')(x)  # SNA, SNB, ANB
    model = models.Model(base_model.input, outputs)
    return model


In [44]:
import cv2
import numpy as np
import pandas as pd
import os

def load_data(image_dir, traced_dir, labels_csv):
    images = []
    traced = []
    labels = pd.read_csv(labels_csv)
    
    for file_name in labels['D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Percision Smile\dataset']:
        img = cv2.imread(os.path.join(image_dir, file_name), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256)) / 255.0
        images.append(img[..., np.newaxis])

        trace = cv2.imread(os.path.join(traced_dir, file_name), cv2.IMREAD_GRAYSCALE)
        trace = cv2.resize(trace, (256, 256)) / 255.0
        traced.append(trace[..., np.newaxis])

    angles = labels[['SNA', 'SNB', 'ANB']].values
    return np.array(images), np.array(traced), np.array(angles)


<>:11: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\C'
C:\Users\VIRAJ GURBADE\AppData\Local\Temp\ipykernel_11208\1683436387.py:11: SyntaxWarning: invalid escape sequence '\C'
  for file_name in labels['D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Percision Smile\dataset']:


In [45]:
# Load the dataset
train_image_dir = r"D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Percision Smile\dataset\train\images"
train_traced_dir = r"D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Percision Smile\dataset\train\traced"
train_labels_csv = r"D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Percision Smile\dataset\train\labels.csv"


test_image_dir = r"dataset\test\images"
test_traced_dir = r"dataset\test\traced"
test_labels_csv = r"dataset\test\labels.csv"

X_train, Y_train, angles_train = load_data(train_image_dir, train_traced_dir, train_labels_csv)
X_test, Y_test, angles_test = load_data(test_image_dir, test_traced_dir, test_labels_csv)

# Build the models
unet_model = build_unet(input_shape=(256, 256, 1))
regression_model = build_regression(input_shape=(256, 256, 1))

# Compile the models
unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
regression_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the tracing (U-Net) model
unet_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=16)

# Train the regression model
regression_model.fit(X_train, angles_train, validation_data=(X_test, angles_test), epochs=10, batch_size=16)


KeyError: 'D:\\COLLEGE\\Projects\\AI-Driven Cephalometric Analysis\\Percision Smile\\dataset'

In [31]:
def load_data(image_dir, traced_dir, labels_csv):
    images = []
    traced = []
    labels = pd.read_csv(labels_csv)
    
    for i, row in labels.iterrows():
        file_name = f"{i+1}.png"  # Adjust this if the naming convention is different
        img_path = os.path.join(image_dir, file_name)
        trace_path = os.path.join(traced_dir, file_name)
        
        print(f"Loading image: {img_path}")  # Debugging line
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"Error loading image: {img_path}")  # Debugging line
            continue  # Skip this image if it's not found or cannot be read
        
        img = cv2.resize(img, (256, 256)) / 255.0
        images.append(img[..., np.newaxis])

        trace = cv2.imread(trace_path, cv2.IMREAD_GRAYSCALE)
        
        if trace is None:
            print(f"Error loading traced image: {trace_path}")  # Debugging line
            continue  # Skip this traced image if it's not found or cannot be read
        
        trace = cv2.resize(trace, (256, 256)) / 255.0
        traced.append(trace[..., np.newaxis])

    angles = labels[['SNA <', 'SNB <', 'ANB <']].values  # Adjust based on the columns in your CSV
    return np.array(images), np.array(traced), np.array(angles)


In [37]:
# Predict traced image
traced_prediction = unet_model.predict(X_test[0:1])[0]
cv2.imshow("Original X-ray", X_test[0])
cv2.imshow("Traced X-ray", traced_prediction)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Predict angles (SNA, SNB, ANB)
predicted_angles = regression_model.predict(X_test[0:1])
print("Predicted SNA, SNB, ANB:", predicted_angles[0])


c:\Users\VIRAJ GURBADE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ValueError: math domain error